In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [2]:
# Inlezen van CSV-bestand
dataPositions = pd.read_csv('../interpolated-data/positions_xyz_128px_full.csv')
dataCoordinates = pd.read_csv('../interpolated-data/red_ball_coordinates_128px_full.csv')
dataPositions.shape, dataCoordinates.shape

((4711, 3), (4711, 14))

In [3]:
combined = pd.concat([dataCoordinates, dataPositions], axis=1)
# remove the rows with to more than 3 NaN values

combined = combined.dropna(thresh=combined.shape[1]-3)
combined

,u0,v0,u1,v1,u2,v2,u3,v3,u4,v4,u5,v5,u6,v6,x,y,z
0,0.0,42.0,87.0,39.0,34.853473,82.531266,44.000000,83.00000,89.000000,83.000000,28.886553,118.791444,69.000000,122.000000,10.138330,3.169981,15.09998
1,0.5,42.5,87.0,40.0,34.853473,82.531266,45.000000,84.00000,89.000000,84.000000,28.886553,118.791444,68.500000,120.500000,10.133340,3.269852,15.19985
2,0.5,43.5,87.0,41.0,34.853473,82.531266,45.000000,85.00000,89.000000,85.000000,28.886553,118.791444,68.500000,120.500000,10.125010,3.369500,15.29950
3,1.0,45.0,87.5,42.0,34.853473,82.531266,45.000000,86.00000,89.000000,86.000000,28.886553,118.791444,68.500000,120.500000,10.113370,3.468816,15.39882
4,1.0,45.5,88.0,43.0,34.853473,82.531266,45.000000,86.00000,90.000000,87.000000,28.886553,118.791444,68.500000,120.500000,10.098430,3.567688,15.49769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4706,122.0,64.0,79.0,115.0,121.000000,117.000000,45.465763,106.74694,84.286003,95.550908,-29.617042,237.188872,90.315662,134.650737,4.720146,3.151380,7.95000
4707,123.0,62.0,96.0,120.5,121.000000,115.000000,45.465763,106.74694,84.286003,95.550908,-29.617042,237.188872,90.315662,134.650737,4.714249,3.084737,7.95000
4708,123.0,61.0,96.0,120.5,121.000000,113.000000,45.465763,106.74694,84.286003,95.550908,-29.617042,237.188872,90.315662,134.650737,4.708378,3.018077,7.95000
4709,123.0,57.0,79.0,115.0,121.000000,109.000000,45.465763,106.74694,84.286003,95.550908,-29.617042,237.188872,90.315662,134.650737,4.702536,2.951475,7.95000


In [4]:
X = combined.iloc[:, :14] # select all rows and columns from 0 to 13
y = combined.iloc[:, 14:] # select x y z output columns

### Data Preprocessing

In [5]:
from sklearn.impute import KNNImputer

#imputer = KNNImputer(n_neighbors=2)
#dfX = imputer.fit_transform(X)
#y = imputer.fit_transform(y)

### Splitting the dataset into the Training set and Test set

In [6]:
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y)


X.shape, y.shape

((4711, 14), (4711, 3))

### Hyperparameter tuning

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor

# Define a parameter grid for the MLPRegressor
param_grid = {
    'model__hidden_layer_sizes': [ (50, 50), (150, 100, 50), (200, 150, 100)],
    'model__activation': ['identity', 'logistic', 'tanh', 'relu'],
    'model__solver': ['sgd', 'adam'],
    'model__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'model__learning_rate_init': [0.001, 0.01, 0.1],
    'model__max_iter': [10000, 20000, 28000],
    'model__early_stopping': [True],
}

# Create a pipeline with an MLPRegressor model
pipeline = Pipeline([
    ('model', MLPRegressor(random_state=42))
])

# Perform a randomized search over the parameter grid
random_search = RandomizedSearchCV(pipeline, param_distributions=param_grid, n_iter=100, cv=5, n_jobs=1, verbose=2, random_state=42)

#### Train the model

In [ ]:
random_search.fit(X_train_scaled, y_train_scaled)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END model__activation=relu, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.001, model__max_iter=28000, model__solver=adam; total time=   3.5s
[CV] END model__activation=relu, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.001, model__max_iter=28000, model__solver=adam; total time=   6.3s
[CV] END model__activation=relu, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.001, model__max_iter=28000, model__solver=adam; total time=  15.2s
[CV] END model__activation=relu, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.001, model__max_iter=28000, model__solver=adam; total time=  16.2s
[CV] END model__a

C:\Users\xande\venv\lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\xande\venv\lib\site-packages\sklearn\neural_network\_base.py:173: RuntimeWarning: overflow encountered in square
  return ((y_true - y_pred) ** 2).mean() / 2
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\neural_network\_base.py:128: RuntimeWarning: invalid value encountered in multiply
  delta *= 1 - Z**2


[CV] END model__activation=tanh, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=constant, model__learning_rate_init=0.1, model__max_iter=28000, model__solver=sgd; total time=   2.3s


C:\Users\xande\venv\lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\Users\xande\venv\lib\site-packages\sklearn\metrics\_regression.py:1005: RuntimeWarning: overflow encountered in square
  numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0, dtype=np.float64)
C:\Users\xande\venv\lib\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


[CV] END model__activation=tanh, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=constant, model__learning_rate_init=0.1, model__max_iter=28000, model__solver=sgd; total time=   2.6s
[CV] END model__activation=tanh, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=constant, model__learning_rate_init=0.1, model__max_iter=28000, model__solver=sgd; total time=   2.5s
[CV] END model__activation=tanh, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=constant, model__learning_rate_init=0.1, model__max_iter=28000, model__solver=sgd; total time=   2.5s


...

[CV] END model__activation=identity, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.1, model__max_iter=20000, model__solver=sgd; total time=   0.1s
[CV] END model__activation=identity, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.1, model__max_iter=20000, model__solver=sgd; total time=   0.1s


C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


[CV] END model__activation=identity, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.1, model__max_iter=20000, model__solver=sgd; total time=   0.0s
[CV] END model__activation=identity, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.1, model__max_iter=20000, model__solver=sgd; total time=   0.0s


C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
C:\Users\xande\venv\lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


[CV] END model__activation=identity, model__early_stopping=True, model__hidden_layer_sizes=(200, 150, 100), model__learning_rate=adaptive, model__learning_rate_init=0.1, model__max_iter=20000, model__solver=sgd; total time=   0.0s
[CV] END model__activation=logistic, model__early_stopping=True, model__hidden_layer_sizes=(50, 50), model__learning_rate=constant, model__learning_rate_init=0.001, model__max_iter=20000, model__solver=adam; total time=  30.0s
[CV] END model__activation=logistic, model__early_stopping=True, model__hidden_layer_sizes=(50, 50), model__learning_rate=constant, model__learning_rate_init=0.001, model__max_iter=20000, model__solver=adam; total time=  33.0s
[CV] END model__activation=logistic, model__early_stopping=True, model__hidden_layer_sizes=(50, 50), model__learning_rate=constant, model__learning_rate_init=0.001, model__max_iter=20000, model__solver=adam; total time=  34.5s
[CV] END model__activation=logistic, model__early_stopping=True, model__hidden_layer_siz

#### Result of the hyperparameter tuning

In [ ]:
model = random_search.best_estimator_

In [ ]:
print('fBest parameters found:\n', random_search.best_params_)

#### Make Predictions

In [ ]:
#y_val_pred_scaled = model.predict(X_val_scaled)
#y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)

#### Evaluate the model

In [ ]:
from utils.evaluate import evaluate_model

#mse_val, mae_val, r2_val = evaluate_model(y_val, y_val_pred, "Validation")

In [ ]:
#y_test_pred_scaled = model.predict(X_test_scaled)
#y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

In [ ]:
#mse_test, mae_test, r2_test = evaluate_model(y_test, y_test_pred, "Test")

#### Visualize the results
The test data is randomized so the plot won't show a clean trajectory

In [ ]:
from utils.plot_maker_gaussian_process import plot_maker_3d

#plot_maker_3d(np.array(y_test)[:10], np.array(y_test_pred)[:10], "Predictions on Test Set")

# Make Predictions on new data

In [ ]:
test_data = pd.read_csv('../interpolated-data/red_ball_coordinates_128px_2024-02-27_13h38m.csv', delimiter=',', decimal='.')
test_result = pd.read_csv('../interpolated-data/positions_xyz_2024-02-27_13h38m.csv', delimiter=',', decimal='.')

In [ ]:
new_combined = pd.concat([test_data, test_result], axis=1)
new_combined = new_combined.dropna(thresh=new_combined.shape[1]-3)
new_combined = new_combined

In [ ]:
new_X = new_combined.iloc[:, :14]
new_y = new_combined.iloc[:, 14:]
new_X.shape, new_y.shape

In [ ]:
#new_X_imputed = imputer.fit_transform(new_X)

new_X_scaled = scaler_X.fit_transform(new_X)
new_y_scaled = scaler_y.fit_transform(new_y)

In [ ]:
new_y_pred_scaled = model.predict(new_X_scaled)

In [ ]:
new_y_pred = scaler_y.inverse_transform(new_y_pred_scaled)

In [ ]:
new_mse, new_mae, new_r2 = evaluate_model(new_y, new_y_pred, "New Data")

In [ ]:
plot_maker_3d(np.array(new_y), np.array(new_y_pred), "Predictions on New Data")